In [101]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import seaborn as sns

In [72]:
dataset = pd.read_csv(r"C:\Users\Ganesh\Downloads\archive (14)\Titanic-Dataset.csv")


In [73]:
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [74]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [75]:
# handling missing values

In [76]:
dataset.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [77]:
dataset["Age"] = dataset["Age"].fillna(dataset['Age'].median())
dataset.drop('Cabin',axis=1, inplace=True)
dataset.drop('Embarked', axis=1, inplace=True)

In [78]:
dataset.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
dtype: int64

In [79]:
# removing non_numerical column 

In [80]:
dataset = dataset.drop(['Name', 'Ticket'], axis=1)

In [81]:
# converting categorical variable into numerical using one-hot encoding 

In [82]:
dataset = pd.get_dummies(dataset, columns=['Sex'])

In [83]:
#train test split
from sklearn.model_selection import train_test_split

In [84]:
#defining features and target variables

In [85]:
X = dataset.drop('Survived', axis=1)
y= dataset['Survived']

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [99]:
#building pipeline 
from sklearn.pipeline import Pipeline

In [102]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier())])

In [103]:
#hyperparameter tuning 

In [104]:
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]}

In [105]:
# using grid search cv for hyper parameter tuning 

In [106]:
from sklearn.model_selection import GridSearchCV

In [108]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

In [111]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [None, 10, 20, 30],
                         'classifier__min_samples_leaf': [1, 2, 4],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [112]:
#getting best model 

In [113]:
best_model = grid_search.best_estimator_

In [114]:
# making prediction

In [115]:
y_pred = best_model.predict(X_test)

In [116]:
# model evaluation 


In [117]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [118]:
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

In [119]:
print(f'Best Model Parameters: {grid_search.best_params_}')
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

Best Model Parameters: {'classifier__max_depth': 30, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100}
Accuracy: 0.8271186440677966
Confusion Matrix:
[[163  12]
 [ 39  81]]
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.93      0.86       175
           1       0.87      0.68      0.76       120

    accuracy                           0.83       295
   macro avg       0.84      0.80      0.81       295
weighted avg       0.83      0.83      0.82       295

